# Add Dependences

In [22]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [23]:
parameters = Model(
    Cubic(
        μ₁      = 0.1,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ      = 0.0000001
    ),
    Time(
        t_f     = 100000, 
        dt      = 0.1
    ),
    Neighbor(
        n_knn   = 50,
        nn      = 12
    ),
    Geometry(
        R_agg   = 9,
        num_agg = 2
    ),
    Simulation(
        n_text  = 200,
        path    = "BBB.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, 2), Simulation(200, "BBB.xyz"))

In [24]:
X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

## Var before func
force(r) =  force_func(parameters.Force::Cubic, r)

force (generic function with 1 method)

In [25]:
function simulate(SAVING::Bool, m::Model, p::F, Agg::Aggregate) where F <: ForceType
    # Definig Variables for calculing initial variables
    global Agg

    # Calculating position of every cell on the fusion
    p = Progress(Int(m.Time.t_f/m.Time.dt),barlen=25)
    for Agg.t in 0:Int(m.Time.t_f/m.Time.dt)

        # Saving data in a given time (n_text)
        if mod(Agg.t, Int(m.Time.t_f/m.Time.dt/m.Simulation.n_text)) == 0 && SAVING
            X_w = Matrix(Agg.Position.X)
            open(m.Simulation.path, "a") do f
                write(f, "$(size(Agg.Position.X, 1))\n")
                write(f, "t=$(Agg.t)\n")
    #             writedlm(f,hcat(X_f, X_w), ' ')
                writedlm(f,hcat(repeat([1.0],size(Agg.Position.X, 1)), X_w), ' ')
            end
        end

        ## LEAPFROG ALGORITHM
        if mod(Agg.t,2) == 0
            if mod(Agg.t, m.Neighbor.n_knn) == 0 
                # Calculating kNN
                cu_knn(Agg)
            end

            # Finding index for random forces in a given time (size(X,1))
            if mod(Agg.t, 2*size(Agg.Position.X,1)) == 0 
                Agg.Neighbor.rand_idx = getindex.(
                    Agg.Neighbor.idx, 1
                    )[getindex.(rand(
                            2:Agg.ParNeighbor.nn,
                            2*size(Agg.Position.X,1)
                        ),1),:]
            end
            # Calculating Forces
            cui_force(m.Time, m.Contractile, Agg)
        else
            Agg.Position.X = Agg.Position.X + Agg.Position.dX * m.Time.dt
        end

        next!(p)
    end
end

simulate (generic function with 2 methods)

In [26]:
simulate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

Progress:   6%|█▌                       |  ETA: 0:20:23m45m

LoadError: InterruptException: